<img src"downloadtime.png">

<h6>Approx 1hr to download both from GDrive</h6>
<img src="download.png">
<h6>Approx 1/2 hr to list barcodes</h6>
<img src="barcode.png">


In [ ]:
import os, sys,subprocess
from subprocess import Popen

first_file="empty_S316_L008_R1_001.fastq.gz"
second_file = "empty_S316_L008_R2_001.fastq.gz"

def uncompress(filename):
    file_handle = os.path.isfile(filename)
    print (file_handle)
    if(file_handle and filename[len(filename)-2:len(filename)]=="gz"):
        print ("filename is gz")
        #test if there is an uncompressed version
        is_uncompressed = os.path.isfile(filename[len(filename)-3:len(filename)])
        if(is_uncompressed):
            print("uncompressed exists")
        else:
            print("uncompressing")
            return_code = subprocess.Popen("gunzip"+" "+filename,shell=True)
            print (return_code) # this doesnt mean the command is complete!
            
uncompress(first_file)
uncompress(second_file) 


In [ ]:
!wc -l empty_S316_L008_R1_001.fastq
#1242098928
!wc -l empty_S316_L008_R2_001.fastq


In [2]:
def process_header(line):
    """
    """
    #print("header:",line)
    
def do_nothing(read_line):
    """
    """
    #print ('do_nothing')


def process_DNA(line):
    """
    """
    #print("DNA:",line)
    for i,j in enumerate(list(line)):
        if(j=='N'):
            #print("DNA bad pos:",i)
            if not i in N_pos_dict:
                N_pos_dict[i]=1
            else:
                N_pos_dict[i] +=1
        #stats for nucleotides
        if not j in nuc_dict:
            nuc_dict[j] = 1
        else:
            nuc_dict[j]+=1
        
        
def process_phred(line):
    """
    """
    #print("quality:",line)
    plot_quality=[ord(x)-33 for x in line]
    for i,j in enumerate (plot_quality):
        if j<20:
            if not i in low_phred:
                low_phred[i]=1
            else:
                low_phred[i] +=1
    #print(plot_quality)
    #plt.plot(plot_quality)
    #plt.show()
    #plt.gcf().clear()
nucleotide_set = {'A','G','T','C','N'}
num_read=0

In [3]:
import matplotlib.pyplot as plt
import time

file_name="empty_S316_L008_R1_001.fastq"


N_pos_dict={}
nuc_dict={}
low_phred={}


num_read = 0;


startTime = time.time()
with open(file_name) as file_open:
    for read_line in file_open:
        if(num_read%100000000==0):
            print(num_read)
        #read_line = readline().strip()
        #print("line num:",num_read,read_line)
        num_read+=1
        if(num_read>0 and num_read<1000):
            function_lookup = {0:(process_phred,read_line),1:(process_header,read_line),2:(process_DNA,read_line),3:(do_nothing,read_line)}
            func = function_lookup.get(num_read%4)
            #print(func[0],func[1])
            func[0](read_line)
endTime=time.time()
print("elpased:",(endTime-startTime))
for i in N_pos_dict:
    print(i,N_pos_dict[i])
print('----------')
for i in nuc_dict:
    print(i,nuc_dict[i])
print('----------')
for i in low_phred:
    print(i,low_phred[i])

0
100000000
200000000
300000000
400000000
500000000
600000000
700000000
800000000
900000000
1000000000
1100000000
1200000000
elpased: 811.894590139389
3 81
39 124
41 58
47 138
149 94
25 30
51 24
48 19
131 15
137 4
146 1
G 6466
T 11856
N 588
C 6540
A 12300

 250
1 14
2 16
3 86
5 4
25 47
30 16
31 28
32 21
39 132
41 74
47 145
48 28
59 5
60 4
62 6
66 14
75 12
77 6
84 12
97 24
98 7
104 12
116 18
117 18
123 15
126 13
129 11
130 15
131 26
132 12
134 14
135 16
137 26
142 22
145 27
147 37
149 125
151 249
13 8
24 16
27 7
29 22
33 26
38 9
45 11
46 14
57 4
58 12
67 10
70 8
73 9
81 9
82 16
86 13
89 15
92 7
101 9
105 20
106 18
121 19
136 45
140 20
141 30
150 60
22 7
37 13
76 10
79 15
94 12
95 7
111 18
113 10
115 9
119 27
120 13
125 22
144 42
146 25
148 31
17 8
23 7
36 18
40 10
51 33
61 7
63 10
64 5
90 9
93 8
107 17
109 14
122 11
127 13
139 31
143 25
34 12
44 10
56 8
68 12
88 13
91 11
128 9
0 10
21 8
26 8
43 14
49 5
85 14
87 6
124 9
4 4
6 2
7 4
14 14
35 16
42 12
53 9
55 11
65 6
69 5
71 8
72 6
96 7
99

In [ ]:
low_phred.values()
low_phred.keys()
#length of read is 151
plt.plot(low_phred.keys(),low_phred.values())
plt.show()
#y = position of read
#x axis number of reads where there is phred < 20. 
#not that bad, only 100-150 reads which we can discard. Is this worth cleaning? out of 1B reads. 
#

In [ ]:
import pickle

def save(obj, name ):
    with open(name, 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load(name ):
    with open( name , 'rb') as f:
        return pickle.load(f)

In [ ]:
save(low_phred,"lp.pkl")
newobj = load("lp.pkl")
print newobj

In [4]:
def process_header(line,num_at):
    """
    process the header 
    """
    #print('process_header, line[0]:',line[0])
    if(line[0]=="@"):
        num_at+=1
        #add more stats, flow cell for where errors are
    else:
        print("error in header")
    
    
def process_plus(read_line,num_plus):
    """
    does nothing but count plus signs for QC
    """
    #print('process_plus')
    if (read_line.strip()=="+"):
        num_plus+=1
    else:
        print("error in plus sign delimiter")


def process_DNA(line):
    """
    """
    #print("process_DNA:")
    for i,j in enumerate(list(line)):
        if(j=='N'):
            #print("DNA bad pos:",i)
            if not i in N_pos_dict:
                N_pos_dict[i]=1
            else:
                N_pos_dict[i] +=1
        #stats for nucleotides
        if not j in nuc_dict:
            nuc_dict[j] = 1
        else:
            nuc_dict[j]+=1
        
        
def process_phred(line):
    """
    """
    plot_quality=[ord(x)-33 for x in line]
    for i,j in enumerate (plot_quality):
        if j<20:
            if not i in low_phred:
                low_phred[i]=1
            else:
                low_phred[i] +=1
    #print(plot_quality)
    #plt.plot(plot_quality)
    #plt.show()
    #plt.gcf().clear()
 

In [8]:
import gzip, time, readline 
import numpy as np

nucleotide_set = {'A','G','T','C','N'}

filename = "empty_S316_L008_R2_001.fastq"

num_read=0
N_pos_dict={}
nuc_dict={}
low_phred={}

startTime = time.time()
num_read = 0;
num_plus = 0
num_at = 0

nucleotide_set = {'A','G','T','C','N'}

with open(filename, 'rb') as file_open:
    for read_line in file_open:
        if(num_read%1000000==0):
            print(num_read)
        try:
            read_line = read_line.decode()
        except AttributeError:
            pass
        read_line = read_line.strip()
        #print("line num:",num_read,read_line)
        if(num_read>=0 ):
            function_lookup = {0:(process_header,read_line,num_at),1:(process_DNA,read_line),2:(process_plus,read_line,num_plus),3:(process_phred,read_line)}
            #print("function_lookup:",function_lookup)
            func = function_lookup.get(num_read%4) #add multiple argouments
            #print("func:",func)
            #execute the functions
            if( (num_read%4)==0):
                func[0](read_line,num_at)
            elif((num_read%4)==2):
                func[0](read_line,num_plus)
            else:
                func[0](read_line)
        num_read+=1 
         
            
endTime=time.time()
print("elpased:",(endTime-startTime))
for i in N_pos_dict:
    print(i,N_pos_dict[i])
for i in nuc_dict:
    print(i,nuc_dict[i])
for i in low_phred:
    print(i,low_phred[i])

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
68000000
69000000
70000000
71000000
72000000
73000000
74000000
75000000
76000000
77000000
78000000
79000000
80000000
81000000
82000000
83000000
84000000
85000000
86000000
87000000
88000000
89000000
90000000
91000000
92000000
93000000
94000000
95000000
96000000
97000000
98000000
99000000
100000000
101000000
102000000
103000000
104000000
105000000
106000000
107000000
108000000
109000000
110000000
111000

830000000
831000000
832000000
833000000
834000000
835000000
836000000
837000000
838000000
839000000
840000000
841000000
842000000
843000000
844000000
845000000
846000000
847000000
848000000
849000000
850000000
851000000
852000000
853000000
854000000
855000000
856000000
857000000
858000000
859000000
860000000
861000000
862000000
863000000
864000000
865000000
866000000
867000000
868000000
869000000
870000000
871000000
872000000
873000000
874000000
875000000
876000000
877000000
878000000
879000000
880000000
881000000
882000000
883000000
884000000
885000000
886000000
887000000
888000000
889000000
890000000
891000000
892000000
893000000
894000000
895000000
896000000
897000000
898000000
899000000
900000000
901000000
902000000
903000000
904000000
905000000
906000000
907000000
908000000
909000000
910000000
911000000
912000000
913000000
914000000
915000000
916000000
917000000
918000000
919000000
920000000
921000000
922000000
923000000
924000000
925000000
926000000
927000000
928000000
929000000
